In [1]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
import project_helper
import project_tests

from tqdm import tqdm

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/miku3090/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/miku3090/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
cik_lookup = {
    'AMZN': '0001018724',
    'BMY': '0000014272',   
}

additional_cik = {
    'AEP': '0000004904',
    'AXP': '0000004962',
    'BA': '0000012927', 
    'BK': '0001390777',
    'CAT': '0000018230',
    'DE': '0000315189',
    'DIS': '0001001039',
    'DTE': '0000936340',
    'ED': '0001047862',
    'EMR': '0000032604',
    'ETN': '0001551182',
    'GE': '0000040545',
    'IBM': '0000051143',
    'IP': '0000051434',
    'JNJ': '0000200406',
    'KO': '0000021344',
    'LLY': '0000059478',
    'MCD': '0000063908',
    'MO': '0000764180',
    'MRK': '0000310158',
    'MRO': '0000101778',
    'PCG': '0001004980',
    'PEP': '0000077476',
    'PFE': '0000078003',
    'PG': '0000080424',
    'PNR': '0000077360',
    'SYY': '0000096021',
    'TXN': '0000097476',
    'UTX': '0000101829',
    'WFC': '0000072971',
    'WMT': '0000104169',
    'WY': '0000106535',
    'XOM': '0000034088'}

In [4]:
sec_api = project_helper.SecAPI()

In [9]:
! pip install beautifulsoup4

  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)


In [20]:
from bs4 import BeautifulSoup

def get_sec_data(cik, doc_type, start=0, count=60):
    newest_pricing_data = pd.to_datetime('2018-01-01')
    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
        '&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom' \
        .format(cik, doc_type, start, count)
    sec_data = sec_api.get(rss_url)
    print('sec_data', sec_data)
    feed = BeautifulSoup(sec_data.encode('ascii')).feed
    print('feed', feed)
    entries = [
        (
            entry.content.find('filing-href').getText(),
            entry.content.find('filing-type').getText(),
            entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry', recursive=False)
        if pd.to_datetime(entry.content.find('filing-date').getText()) <= newest_pricing_data]

    return entries

In [5]:
from bs4 import BeautifulSoup

def get_sec_data_stored(sec_data_path, doc_type):
    newest_pricing_data = pd.to_datetime('2018-01-01')
    with open(sec_data_path, 'r') as f:
        sec_data = f.read()
    feed = BeautifulSoup(sec_data.encode('ascii')).feed
#     print(feed)
    entries = [
        (
            entry.content.find('filing-href').getText(),
            entry.content.find('filing-type').getText(),
            entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry', recursive=False)
        if pd.to_datetime(entry.content.find('filing-date').getText()) <= newest_pricing_data]

    return entries


In [6]:
example_ticker = 'AMZN'
sec_data_path = 'sec_feed.txt'
sec_data = {}
sec_data[example_ticker] = get_sec_data_stored(sec_data_path, '10-K')
pprint.pprint(sec_data[example_ticker][:5])
# pprint.pprint(sec_data[example_ticker][:5])

[('https://www.sec.gov/Archives/edgar/data/1018724/000101872417000011/0001018724-17-000011-index.htm',
  '10-K',
  '2017-02-10'),
 ('https://www.sec.gov/Archives/edgar/data/1018724/000101872416000172/0001018724-16-000172-index.htm',
  '10-K',
  '2016-01-29'),
 ('https://www.sec.gov/Archives/edgar/data/1018724/000101872415000006/0001018724-15-000006-index.htm',
  '10-K',
  '2015-01-30'),
 ('https://www.sec.gov/Archives/edgar/data/1018724/000101872414000006/0001018724-14-000006-index.htm',
  '10-K',
  '2014-01-31')]


/home/miku3090/anaconda3/envs/ai_trading_nlp/lib/python3.6/site-packages/bs4/builder/__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  XMLParsedAsHTMLWarning.MESSAGE, XMLParsedAsHTMLWarning


In [7]:
file_url = 'https://www.sec.gov/Archives/edgar/data/1018724/000101872417000011/0001018724-17-000011.txt'

In [8]:
with open('sec_10ks.txt', 'r') as f:
    res = f.read()

In [9]:
import re


def get_documents(text):
    """
    Extract the documents from the text

    Parameters
    ----------
    text : str
        The text with the document strings inside

    Returns
    -------
    extracted_docs : list of str
        The document strings found in `text`
    """
    
    # TODO: Implement
    final_docs = []
    start_regex = re.compile(r'<DOCUMENT>')
    end_regex = re.compile(r'</DOCUMENT>')
    
    start_idx = [x.end() for x in re.finditer(start_regex, text)]
    end_idx = [x.start() for x in re.finditer(end_regex, text)]
    
    for start_i, end_i in zip(start_idx, end_idx):
        final_docs.append(text[start_i:end_i])
    
    
    return final_docs

In [18]:
doc_res = get_documents(res)
print(doc_res[5])


<TYPE>EX-31.2
<SEQUENCE>6
<FILENAME>amzn-20161231xex312.htm
<DESCRIPTION>SECTION 302 CFO CERTIFICATION
<TEXT>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
	<head>
		<!-- Document created using Wdesk 1 -->
		<!-- Copyright 2017 Workiva -->
		<title>Exhibit</title>
	</head>
	<body style="font-family:Times New Roman;font-size:10pt;">
<div><a name="s75FABEEA3E4C5F7189926BA2A043DB33"></a></div><div><div style="line-height:120%;font-size:10pt;"><font style="font-family:inherit;font-size:10pt;"><br></font></div></div><div><br></div><div style="line-height:120%;text-align:right;font-size:10pt;"><font style="font-family:inherit;font-size:10pt;font-weight:bold;">Exhibit 31.2</font></div><div style="line-height:120%;padding-top:12px;text-align:center;font-size:10pt;"><font style="font-family:inherit;font-size:10pt;font-weight:bold;">CERTIFICATIONS</font></div><div style="line-height:120%;padding-top:18px;text-align:left;font-size:1

In [11]:
def remove_html_tags(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    return text


def clean_text(text):
    text = text.lower()
    text = remove_html_tags(text)
    
    return text